# Import the dataset and explore the data

In [2]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
import statistics

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Seeting seaborn style
sns.set()

In [ ]:
wcb = pd.read_csv('train_data.csv', sep = ',')   #sep is good to seperate data
pd.set_option('display.max_columns', None) #to be able too see all columns
wcb.head(5)

# Data Exploration

#### Shape

In [ ]:
wcb.shape

In [ ]:
wcb.info()

### Any footnotes?

In [ ]:
wcb.tail(5)
# No, there isn't any

### Check for missing values, strange values and datatypes


#### Check for disposable rows or columns (with NaN, 0)


##### For numerical data
Columm: OIICS Nature of Injury Description <- drop <br>

##### For all data
Rows: 19445 (and 14 columns with this exact number of rows w missing values) <- drop

In [ ]:
wcb.info()

In [ ]:
#wcb = wcb.apply(lambda col: pd.to_numeric(col, downcast='integer', errors='coerce') if col.dtype == 'float' else col)

In [ ]:
wcb_float_to_int = ['Age at Injury', 'Birth Year', 'IME-4 Count', 'Industry Code', 
                    'Agreement Reached', 'WCIO Cause of Injury Code', 
                    'WCIO Nature of Injury Code', 'WCIO Part Of Body Code', 
                    'Number of Dependents']

for col in wcb_float_to_int:
# Convert the column to a nullable integer type (Int64) while keeping NaNs
    wcb[col] = pd.to_numeric(wcb[col], errors='coerce').astype('Int64')

In [ ]:
# transformar as datas em mês e ano
wcb['Accident Date'] = pd.to_datetime(wcb['Accident Date'], errors='coerce')

# Extract year and month, and combine them into a new column
wcb['Accident Year-Month'] = wcb['Accident Date'].dt.to_period('M')

# Display the new column
print(wcb['Accident Year-Month'])

In [ ]:
wcb = wcb[(wcb['Accident Date'].dt.year >= 2020) & (wcb['Accident Date'].dt.year <= 2022)]

In [ ]:
wcb


Check columns that sum to 0:

In [ ]:
wcb.select_dtypes(include='number').sum()

Drop column that sums to 0:

In [ ]:
wcb.drop(columns = ['OIICS Nature of Injury Description'], inplace = True)

In [ ]:
wcb.info()

In [ ]:
wcb[['WCIO Part Of Body Code', 'WCIO Part Of Body Description']] # rever depois de fazer a correlação
# ver também 21 a 26

Check how many missing values are there per column:

In [ ]:
wcb.isna().sum()

In [ ]:
#First we ignore the 2 columns with no missing values - since these are identifying for cases, but don't add much info about each case
nan_columns = wcb.columns.drop(['Assembly Date', 'Claim Identifier'])

# Count how many rows have all NaN values in the selected columns
((wcb[nan_columns].isnull()) | (wcb[nan_columns] == 0)).all(axis=1).sum()

In [ ]:
# selecionar todas as colunas que têm valores nulos, dar drop a rows caso todas essas variaveis fossem nulas
# eliminar targets nulos, que acabou por ser feito indiretamente neste passo
wcb = wcb.dropna(subset = nan_columns, how = 'all')

In [ ]:
wcb.shape
# We know have 574026 rows, less 19445 then before

In [ ]:
#Check missing values again, after first clean-up
wcb.isnull().sum()

In [ ]:
non_null_percentage = wcb.notna().mean() * 100

# Convert it to a DataFrame for better display
non_null_df = non_null_percentage.reset_index()
non_null_df.columns = ['Variable', 'Non-Null Percentage']
non_null_df.sort_values('Non-Null Percentage', ascending=False)

In [ ]:
wcb[wcb['WCIO Part Of Body Code'] == -9] 

In [ ]:
# transformar o -9 em 9, deve ter sido um erro

In [ ]:
wcb['IME-4 Count']

In [ ]:
wcb['C-3 Date'] = wcb['C-3 Date'].apply(lambda x: 0 if pd.isna(x) else 1)

In [ ]:
wcb['First Hearing Date'] = wcb['First Hearing Date'].apply(lambda x: 0 if pd.isna(x) else 1)

In [ ]:
wcb['IME-4 Count']

In [ ]:
non_null_percentage = wcb.notna().mean() * 100

# Convert it to a DataFrame for better display
non_null_df = non_null_percentage.reset_index()
non_null_df.columns = ['Variable', 'Non-Null Percentage']
non_null_df.sort_values('Non-Null Percentage', ascending=False)

### Proposal to fill missing values (for numerical data)
`Age at Injury`                        can be replaced with median <br>
`Average Weekly Wage`                   can be replaced with the average weekly age per age for the rows with values <br>
`Birth Year`                            if there is age, 2024-age at injury <br>
`IME-4 Count`                           Number of IME-4 forms received per claim. The IME-4 form is 
the “Independent Examiner's Report of Independen  Medica 
Examination” for m.missing values = 0 <br>
`Industry Code`                         <> NaN. if industry description isn't NaN, we can check code from this column <br>
`OIICS Nature of Injury Description`  <> oiics manual 2007 downloaded   <br>
`WCIO Cause of Injury Code`           site blocked   <br>
`WCIO Nature of Injury Code`          site blocked   <br>
`WCIO Part Of Body Code`              site blocked <br>r>
`Agreement Reacd `   Binary value: Yes if there is an agreement without t heinvolvement of the WCB -> "unknown:n at th  start of a clai -> shou d return 0 or 1 m<b>

`Number of Dependents`                  this variable is not explained.  NaN we can assume it's 0 dependents <br>

dtype: float64

### Datatypes:

`Age at Injury` float -> int <br>
`Birth Year` float -> int <br>
`IME-4 Count` -> int <br>
`Industry Code` -> int <br>
`Agreement Reached` -> int <br>
`WCIO Cause of Injury Code` -> int <br>
`WCIO Nature of Injury Code` -> int <br>
`WCIO Part Of Body Code` -> int? there are negative values! <br>
`Number of Dependents` -> int <br>
`C-3 Date` -> transformar numa variavel binaria <br>
`First Hearing Date`	-> transformar numa variavel binaria <br>

#### Notes

`WCIO Part Of Body Code` -> there are negative values!!! <br>
Dados apenas de 2020 ate 2022 <br>
Transformar a data em anos e meses <br>
Outliers nas idades (boxplot) <br>

In [ ]:
print(wcb.columns.tolist())

#### Change datatypes

In [ ]:
# Fill null values with 0 otherwise there will be error bc we can't convert non-finite numbers to integers

wcb['Age at Injury'] = wcb['Age at Injury'].astype(int)
wcb['IME-4 Count'] = wcb['IME-4 Count'].fillna(0).astype(int)
wcb['Industry Code'] = wcb['Industry Code'].fillna(0).astype(int)
wcb['Agreement Reached'] = wcb['Agreement Reached'].fillna(0).astype(int)
wcb['WCIO Cause of Injury Code'] = wcb['WCIO Cause of Injury Code'].fillna(0).astype(int)
wcb['WCIO Nature of Injury Code'] = wcb['WCIO Nature of Injury Code'].fillna(0).astype(int)
wcb['WCIO Part Of Body Code'] = wcb['WCIO Part Of Body Code'].fillna(0).astype(int)
wcb['Number of Dependents'] = wcb['Number of Dependents'].fillna(0).astype(int)

### Drop Duplicates

In [ ]:
wcb.duplicated().sum()
#There is none to drop

In [ ]:
duplicate_rows = wcb.duplicated()

# If you want to return the entire DataFrame with only the duplicate rows:
duplicate_rows_df = wcb[duplicate_rows]

duplicate_rows_df

### Choose column for index
`Claim Identifier` ->returns unique values and there is one identifying value for each row/case

In [ ]:
#Check which unique values there are for this column and store as variable
id_claim = wcb['Claim Identifier'].unique()
#count all unique values and see if its the same number as total rows
len(id_claim) == len(wcb)

In [ ]:
wcb = wcb.set_index('Claim Identifier')

## Categorical Data
Understand unique values <br>

`Accident Date` <br>
`Alternative Dispute Resolution` <br>
`Assembly Date` <br>`'Attorney/Representativ` <br>
`'C-2 Dat`, <br>
`'C-3 Dat` <br>,
`'Carrier Nam` <br>,` 'Carrier Ty` <br>'
` 'Claim Injury Ty` <br>'
` 'County of Inju` <br>'`  'COVID-19 Indica` <br>r
`, 'District N` <br>e
`, 'First Hearing D` <br>e
`, 'Gen` <br>
   
`   'Industry Code Descrip` <br>o
`', 'Medical Fee Re` <br>o`    'WCIO Cause of Injury Descri` <br>i
`n', 'WCIO Nature of Injury Descri` <br>i`     'WCIO Part Of Body Descr` <br>t
`on', 'Zi`  <br>C
`de', 'WCB De` <br>ision'Decision'

### Notes
`WCB Decision` is always 'not work related' <br>
`Gender`returns M, F, Y, U -> what does Y and U stand for?

In [ ]:
categorical = wcb.select_dtypes(include=['object', 'category'])
categorical.columns

In [ ]:
wcb.describe(include=['object', 'category']).T

In [ ]:
unique = wcb.select_dtypes(include=['object', 'category']).apply(lambda x: x.unique(), axis=0)
pd.set_option('display.max_colwidth', 100)  # No limit on column width
unique

In [ ]:
wcb['WCB Decision'].unique()

In [ ]:
wcb['Alternative Dispute Resolution'].unique()

In [ ]:
wcb['Carrier Name'].unique()

In [ ]:
wcb['Gender'].unique()

In [ ]:
wcb['Medical Fee Region'].unique()

In [ ]:
wcb['District Name'].unique()

In [ ]:
wcb['Claim Injury Type'].unique()

In [ ]:
wcb['WCB Decision'].unique()

### Make sure the metrics are reasonable numbers, depending on the context of each variable
Especially mean, max, min

In [ ]:
wcb.describe(include='number').T

# Aggregations to comprehend data

In [ ]:
wcb.columns

In [ ]:
wcb.groupby(['Claim Injury Type','Gender']).agg({'Average Weekly Wage':['mean', 'min', 'max']})

In [ ]:
wcb.groupby(['Claim Injury Type','District Name']).agg({'Age at Injury':['mean', 'min', 'max'], 'Average Weekly Wage':['mean', 'min', 'max']})

In [ ]:
wcb.groupby('Age at Injury').agg({'Number of Dependents':['mean', 'min', 'max']})

# Split of data

## Fit and scaling



# Visualizations
for outliers analysis, and more in-depth conclusions

## Day of the year (date) - needs improvement

In [ ]:
wcb['Day of Year'] = wcb['Accident Date'].dt.dayofyear

In [ ]:
# Map Gender to numerical values (e.g., 0 for Male, 1 for Female)
wcb['Gender Numeric'] = wcb['Gender'].map({'M': 0, 'F': 1})

In [ ]:
# Create the scatter plot
plt.figure(figsize=(12, 6))
plt.scatter(wcb['Day of Year'], wcb['Gender Numeric'], alpha=0.6, color='skyblue')

# Add labels and title
plt.title('Scatter Plot of Accident Date vs Gender', fontsize=14)
plt.xlabel('Day of the Year (Accident Date)', fontsize=12)
plt.ylabel('Gender (0=M, 1=F)', fontsize=12)

# Customizing the y-ticks for better interpretation
plt.yticks([0, 1], ['M', 'F'])

# Show the plot
plt.show()

## Age at Injury

In [ ]:
# Define age categories
bins = [0, 17, 30, 40, 50, 60, 74, 100]
labels = ['0-17', '18-30', '31-40', '41-50', '51-60', '61-74', '75+']

# Create a new column for age categories
wcb['Age Category'] = pd.cut(wcb['Age at Injury'], bins=bins, labels=labels, right=False)

In [ ]:
# Count the number of occurrences in each age category
age_distribution = wcb['Age Category'].value_counts()

In [ ]:
# Plot the pie chart
plt.figure(figsize=(7, 7))  # Set the figure size

age_distribution.plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'lightgreen', 'coral', 'orange', 'purple', 'yellow', 'pink'])

# Customize the plot
plt.title('Age Distribution of Accidents')
plt.ylabel('')  # Remove the default y-label (which shows "Age Category")

# Show the plot
plt.show()

In [ ]:
wcb['Age at Injury'].plot(kind='box')

In [ ]:
statistics.variance(wcb['Age at Injury'].dropna())

## Histograms of all variables

In [ ]:
# Step 2: Loop through each column (variable) and create a histogram
for i in range(len(wcb.columns)):
    column_name = wcb.columns[i]  # Get the column name
    plt.figure(figsize=(8, 5))  # Optional: Set figure size for better visuals
    plt.hist(wcb[column_name].dropna(), bins=30, edgecolor='k', alpha=0.7)  # Drop NaNs and create the histogram
    plt.title(f'Histogram of {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.grid(True)  # Optional: Add grid
    plt.show()  # Display the plot

In [ ]:
wcb['Gender'].value_counts().plot(kind='bar')

In [ ]:
wcb['Alternative Dispute Resolution'].value_counts().plot(kind='bar')

## Correlation Matrix

In [ ]:
# Step 1: Select only numerical columns
wcb = wcb.drop('OIICS Nature of Injury Description', axis=1)
numerical_df = wcb.select_dtypes(include='number')

# Step 2: Calculate the correlation matrix
correlation_matrix = numerical_df.corr()

# Step 3: Visualize the correlation matrix
plt.figure(figsize=(10, 8))  # Optional: Adjusts the size of the plot
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)

# Display the plot
plt.title('Correlation Matrix (Numerical Variables Only)')
plt.show()